In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
from scipy.stats import gumbel_r

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def collection_distribution_hev(n,collection,price):
    # generate mean 
    ''' mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1) '''
    # generate deterministic utilities for products
    
    rho = -0.5 # prices and utilities are negatively correlated
    price_mean = np.mean(price[1:]) 
    price_std = np.std(price[1:])
    
    z = np.zeros(len(price[1:]))
    for i in range(len(z)):
        z[i] = (price[1:][i] - price_mean)/price_std
        
    mu_1n = np.zeros(len(z))
    normal_rvs = np.random.randn(len(mu_1n))
    for i in range(len(mu_1n)):
        mu_1n[i] = rho*z[i] + np.sqrt(1-rho**2)*normal_rvs[i] 
    ## deterministic utility of the outside option is strictly greater than the utilities of the products
    mu_0 = np.max(mu_1n) + 1
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = 10
    scale_1n = np.random.uniform(0.04,1,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])
    # generate covariance matrix with positive correlation
    # neg_cov = generate_negatively_correlated_covariance_matrix(n)
    # if is_positive_semidefinite(neg_cov): 
    #     sample_size = 10000
    #     utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    # else:
    #     print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [5]:
''' def collection_distribution_hev(n,collection):
    # generate mean 
    mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1)
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = np.random.uniform(9,10)
    scale_1n = np.random.uniform(0.04,2,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])

    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution '''

' def collection_distribution_hev(n,collection):\n    # generate mean \n    mu_0 = np.random.uniform(2,3)\n    mu_1n = np.random.uniform(-3,-2,n-1)\n    # np.random.uniform(-2,2,n-1)\n    mu = np.hstack((mu_0,mu_1n))\n    \n    scale_0 = np.random.uniform(9,10)\n    scale_1n = np.random.uniform(0.04,2,n-1)\n    scales = np.hstack((scale_0,scale_1n))\n    \n    sample_size = 10000\n    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])\n\n    \n    collection_distribution = np.zeros((n,len(collection)))\n    for i in range(len(collection)):\n        curr_assortment = collection[i]\n        curr_population = [[] for _ in range(sample_size)] \n\n        for j in range(sample_size):\n            for k in curr_assortment:\n                curr_population[j].append(utility_samples[j][k])\n                # each sub list records only the utilities of products in the current assortment\n                \n        frequency = [0]*len(curr_assortment)\n  

In [6]:
def whole_instance_generation(n,whole_collection,whole_offertimes,price):
    ## generate LCMNL instances based on the assortment collection infomation
    # step 1: randomly generate LCMNL parameters
    # 1.1 the number of classes in LCMNL, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mnl
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCMNL instance with the above parameters of LCMNL
    true_hev_instance = collection_distribution_hev(n,whole_collection,price)
    # step 3: generate multinomial samples based on lcmnl instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_hev_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [7]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [8]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [9]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [10]:
# check if the collection are nested
def check_subsets(lists):
    # Convert all lists to sets
    sets = [set(lst) for lst in lists]
    
    # Check subsets
    for i in range(len(sets) - 1):
        if not sets[i + 1].issubset(sets[i]):
            return False
    return True

# Check and print the result
result = check_subsets(all_full_assortment_index)
print("All subsequent lists are subsets of the previous one:", result)

All subsequent lists are subsets of the previous one: True


In [11]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [12]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])
    
    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    #print(pd.DataFrame(curr_whole_instance))
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-6 or curr_choice_collection[y][x] >0.999:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-6 or curr_choice_collection[y][x] >0.999:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:  
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples)    '''

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [13]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [14]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        ''' condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-3:
                    condition = True
        if condition == True:
            print('instance_generation error') '''
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [15]:
# train-test split
all_train_instances = []
all_train_collection = []
all_test_instance = []
all_test_collection = []
all_train_lb = []
all_train_ub = []
all_train_offertimes = []

for i in range(len(pred_instance_size)):
    train_instances_collection = []
    train_collection = []
    train_lb_collection = []
    train_ub_collection = []
    train_offertimes_collection = []
    
    test_instance_collection = []
    test_collection = []
    
    test_index_collection = []
    
    
    for j in range(pred_instance_size[i]):
        
        curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
        curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
        curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
        
        curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
        
        curr_train_collection = []
        curr_test_collection = []
        curr_offertimes_lst = []
        
        while(len(curr_train_collection)==0):
            
            chosen_test_idx = random.sample(range(len(all_full_collections[i])),pred_test_collection_size[i])
            chosen_test_idx.sort()
            
            chosen_train_idx = [x for x in range(len(all_full_collections[i])) if x not in chosen_test_idx ]
    
            for k in range(len(chosen_train_idx)):
                curr_train_collection.append(all_full_collections[i][chosen_train_idx[k]])
                curr_train_instance[:,k] = all_full_instances[i][j][:,chosen_train_idx[k]]
                curr_train_lb[:,k] = all_full_lb[i][j][:,chosen_train_idx[k]]
                curr_train_ub[:,k] = all_full_ub[i][j][:,chosen_train_idx[k]]
                
                curr_offertimes_lst.append(all_full_offertimes[i][chosen_train_idx[k]])
                
            for k in range(len(chosen_test_idx)):
                curr_test_collection.append(all_full_collections[i][chosen_test_idx[k]])
                curr_test_instance[:,k] = all_full_instances[i][j][:,chosen_test_idx[k]]
                
            ## check if the training instance includes all the testing products in the test instance
            # ensure each test product has been offered at least  once
            new_lst_train = sum(curr_train_collection,[])
            unique_numbers_train = list(set(new_lst_train)) 
            
            new_lst_test = sum(curr_test_collection,[])
            unique_numbers_test = list(set(new_lst_test)) 
            
            for num in unique_numbers_test:
                if num not in unique_numbers_train:
                    #print('the unique train numbers',unique_numbers_train)
                    #print('the unique test numbers',unique_numbers_test)
                    curr_train_collection = []
                    curr_test_collection = []
                    curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                    curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                    curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                    curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                    curr_offertimes_lst = []
            if chosen_test_idx in test_index_collection:
                #print('check if exisits repeated indexes ')
                curr_train_collection = []
                curr_test_collection = []
                curr_train_instance = np.zeros((n,pred_train_collection_size[i]))
                curr_test_instance = np.zeros((n,pred_test_collection_size[i]))
                curr_train_lb = np.zeros((n,pred_train_collection_size[i]))
                curr_train_ub = np.zeros((n,pred_train_collection_size[i]))
                curr_offertimes_lst = []
            elif len(curr_test_collection)>0:
                test_index_collection.append(chosen_test_idx)
        #print('chosen training indexes',chosen_train_idx)
        #print('chosen testing indexes',chosen_test_idx)
        #print('curr train collection\n',curr_train_collection)
        #print('curr test collection\n',curr_test_collection)
        df_train = pd.DataFrame(curr_train_instance)
        df_test = pd.DataFrame(curr_test_instance)
        df_offertimes = pd.DataFrame(curr_offertimes_lst)
        #print('curr train instance\n',df_train)
        #print('curr test instance\n',df_test)
        
        ## output the training and testing instance
        ''' df_train.to_csv('instances/train_instances/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_test.to_csv('instances/test_instances/test_'+str(pred_offer_times_list[i])+'/test_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        df_offertimes.to_csv('instances/train_offertimes/train_'+str(pred_offer_times_list[i])+'/train_offertimes'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv') '''
        
        train_instances_collection.append(curr_train_instance)
        train_collection.append(curr_train_collection)
        train_lb_collection.append(curr_train_lb)
        train_ub_collection.append(curr_train_ub)
        train_offertimes_collection.append(curr_offertimes_lst)
                
        test_instance_collection.append(curr_test_instance)
        test_collection.append(curr_test_collection)
        
    all_train_instances.append(train_instances_collection)
    all_train_collection.append(train_collection)
    all_train_lb.append(train_lb_collection)
    all_train_ub.append(train_ub_collection)
    all_train_offertimes.append(train_offertimes_collection)
    all_test_instance.append(test_instance_collection)
    all_test_collection.append(test_collection)    
        

In [16]:
def row_assortment_data(data,collection):
    pre_data = copy.deepcopy(data)
    
    for j in range(data.shape[0]):
        for i in range(len(collection)):
            if j not in collection[i]:
                pre_data[j][i] = -1
    
    return pre_data

def find_indexes_nonnegative_descending(arr):
    row_indexes_sorted_desc_nonnegative = []
    row_indexes_positive_same_value = []

    for row in arr:
        # Find indexes with non-negative values
        nonnegative_row = row[row >= 0]  # Consider only non-negative values
        unique_elements, unique_indexes = np.unique(nonnegative_row, return_index=True)
        sorted_indexes_desc = np.nonzero(row >= 0)[0][unique_indexes][np.argsort(-unique_elements)]

        row_indexes_sorted_desc_nonnegative.append(sorted_indexes_desc)

        # Keep all indexes of elements with positive and same values
        positive_values = unique_elements[unique_elements > 0]
        positive_same_value_indexes = [np.where(row == val)[0] for val in positive_values if np.count_nonzero(row == val) > 1]
        row_indexes_positive_same_value.append(positive_same_value_indexes)

    return row_indexes_sorted_desc_nonnegative, row_indexes_positive_same_value

In [17]:
def mdm_feasibility(data,ranking,equal):
      
    ub = 100

    model = gp.Model('mdm_feasibility')
    model.setParam('OutputFlag', 0)
    eps = model.addVar(name='eps')
    lam = model.addVars(data.shape[1],name = 'lam')
    
    model.addConstr(eps<=ub ) # just give an upper bound (+ve) for eps, o.w., the problem is unbounded
    
    #for i in range(ranking.shape[0]): # from each product 
    for i in range(len(ranking)): # from each product 
        for j in range(len(ranking[i])-1): # the ranking of assortment in product i 
            model.addConstr(lam [ranking[i][j+1]] - lam[ranking[i][j]]-  eps >=0)
    
    for i in range(len(equal)):
        if len(equal[i])>0:
            for j in range(len(equal[i][0])-1):
                model.addConstr(lam[equal[i][0][j]]-lam[equal[i][0][j+1]]==0)
    
    model.setObjective(eps,GRB.MAXIMIZE)
    model.optimize()
    
     # Access the optimal solution
    lam_values = [lam[i].x for i in range(data.shape[1])]
    lam_values = np.array(lam_values)

    if (model.objVal> 0):
        #model.write("mdmfeas_out.sol")
        #model.write("mdmfeas_model.lp") 
        return [1,model.Runtime,lam_values]
    else:
        #model.write("mdminfeas_model.lp")
        #return [0, used_time]
        return [0,model.Runtime,lam_values]
    

In [18]:
def mdm_limit(data, collection,offer_times):
    n, m = data.shape

    prod_in_ass = [[] for _ in range(n)]
    for i, assort in enumerate(collection):
        for j in assort:
            prod_in_ass[j].append(i)

    eps = 0.001

    model = gp.Model('mdm_limit')
    model.setParam('OutputFlag', 0)
    lam = model.addVars(m, vtype=GRB.CONTINUOUS, lb=0.0, name="lam")
    x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="x")
    delta = model.addVars(m, m, vtype=GRB.BINARY, name="delta")
    
    # Define auxiliary binary variables for absolute value terms
    abs_vars = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0.0, name="abs")

    # Add constraints
    for j in range(m):
        model.addConstr(sum(x[i, j] for i in range(n)) == 1, name=f"normalization_{j}")

    for i in range(len(collection)):
        for j in range(n):
            if j not in collection[i]:
                model.addConstr(x[j, i] == 0, name=f"forcing_zero_{i}_{j}")
    
    for i in range(m):
        model.addConstr(lam[i] <= 1, name=f"lam_bound_{i}")

    for i in range(len(prod_in_ass)):
        if len(prod_in_ass[i]) > 0:
            for j in prod_in_ass[i]:
                for k in prod_in_ass[i]:
                    if j != k:
                        model.addConstr(lam[j] - lam[k] + delta[j, k] >= 0, name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(lam[j] - lam[k] - 1 + delta[j, k] + eps * delta[j, k] <= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] + 1 >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - 1 + delta[k, j] <= 0,
                                        name=f"pro_{i}_aspair_{i}_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] + delta[j, k] + delta[k, j] >= 0,
                                        name=f"pro_{i}_aspair_{j}_{k}")
                        model.addConstr(x[i, j] - x[i, k] - delta[j, k] - delta[k, j] <= 0,
                                        name=f"ppro_{i}_aspair_{j}_{k}")

    # Add linear constraints to represent absolute value terms
    for i in range(n):
        for j in range(m):
            model.addConstr(x[i, j] - data[i, j] <= abs_vars[i, j], name=f"abs_const1_{i}_{j}")
            model.addConstr(-x[i, j] + data[i, j] <= abs_vars[i, j], name=f"abs_const2_{i}_{j}")

    # Define the modified objective function using auxiliary variables
    obj = (sum(offer_times[j]*data[i, j] * abs_vars[i, j] for i in range(n) for j in range(m)))
    #obj = (sum(offer_times[j]*data[i][j] * abs_vars[i, j] for i in range(n) for j in range(m))/sum(offer_times))

    model.setObjective(obj, GRB.MINIMIZE)

    # Set Gurobi parameters if needed
    model.optimize()
    
    if model.Status == GRB.OPTIMAL:
        x_values = [[x[i, j].X for j in range(m)] for i in range(n)]
        x_values = np.array(x_values)
        
        lam_values = [lam[i].x for i in range(m)]
        lam_values = np.array(lam_values)
        return [model.objVal, model.Runtime,lam_values,x_values]
    else:
        print('mdm limit computation error')
        #model.write("mdmlimit_out.sol")
        #model.write("mdmlimit_model.lp") 
        #return [model.objVal, model.Runtime,x_values]

In [19]:
def kendall_tau_distance(values1, values2):
    """Compute the Kendall tau distance."""
    num = len(values1)
    assert len(values2) == num, "Both lists have to be of equal length"
    i, j = np.meshgrid(np.arange(num), np.arange(num))
    a = np.argsort(values1)
    b = np.argsort(values2)
    ndisordered = np.logical_or(np.logical_and(a[i] < a[j], b[i] > b[j]), np.logical_and(a[i] > a[j], b[i] < b[j])).sum()
    return ndisordered/2 

In [20]:
def robust_prediction_lp(lambda_S_lst,lambda_A,prod_in_ass,assortment_A,limit_prob,price):
    # robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
    
    model = gp.Model('robust_rev_lp')
    model.setParam('OutputFlag', 0)
    x = model.addVars(len(assortment_A),vtype=GRB.CONTINUOUS, lb=0, name='x')
    
    model.addConstr(gp.quicksum(x[i] for i in range(len(assortment_A))) == 1, name="Normalization")
    
    ## MDM consistency
    for i in range(len(assortment_A)):
        for j in range(len(prod_in_ass[assortment_A[i]])):
            if lambda_A >= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]<= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
            if lambda_A <= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]>= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
    
    # Objective
    obj = gp.quicksum(price[assortment_A[i]] * x[i] for i in range(len(assortment_A)))
    model.setObjective(obj, GRB.MINIMIZE)

    # Optimize the model
    model.optimize()
    
    if model.Status == GRB.OPTIMAL :
        return [model.objVal, [x[i].x for i in range(len(assortment_A))], model.Runtime]
    else:
        return [100000,[-1 for i in range(len(assortment_A))], model.Runtime]
    
    

In [21]:
def best_case_prediction_lp(lambda_S_lst,lambda_A,prod_in_ass,assortment_A,limit_prob,price):
    
    model = gp.Model('best_case_rev_lp')
    model.setParam('OutputFlag', 0)
    x = model.addVars(len(assortment_A),vtype=GRB.CONTINUOUS, lb=0, name='x')
    
    model.addConstr(gp.quicksum(x[i] for i in range(len(assortment_A))) == 1, name="Normalization")
    
    ## MDM consistency
    for i in range(len(assortment_A)):
        for j in range(len(prod_in_ass[assortment_A[i]])):
            if lambda_A >= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]<= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
            if lambda_A <= lambda_S_lst[prod_in_ass[assortment_A[i]][j]]:
                model.addConstr(x[i]>= limit_prob[ assortment_A[i]][prod_in_ass[assortment_A[i]][j]])
    
    # Objective
    obj = gp.quicksum(price[assortment_A[i]] * x[i] for i in range(len(assortment_A)))
    model.setObjective(obj, GRB.MAXIMIZE)

    # Optimize the model
    model.optimize()
    
    if model.Status == GRB.OPTIMAL :
        return [model.objVal, [x[i].x for i in range(len(assortment_A))], model.Runtime]
    else:
        return [-100000,[-1 for i in range(len(assortment_A))], model.Runtime]
    
    

In [22]:
## mdm milp for the worst case revenue
def mdm_revenue(curr_limit_prob, curr_lam, prod_in_ass,lambda_A_lst, assortment_A, price):
    
    robust_revenue_lst = []
    best_case_revenue_lst = []
    robust_pred_runtime = 0
    best_case_pred_runtime = 0
    #robust_prob_lst = []
    #best_case_prob_lst = []

    for j in range(len(lambda_A_lst)):
        lambda_A = lambda_A_lst[j]
        curr_robust_revenue,curr_robust_prob,curr_robust_runtime = robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
        
        robust_revenue_lst.append(curr_robust_revenue)
        robust_pred_runtime = robust_pred_runtime + curr_robust_runtime
        #robust_prob_lst.append(curr_robust_prob)
        
        curr_best_case_revenue,curr_bestcase_prob,curr_best_case_runtime = best_case_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,price)
        best_case_revenue_lst.append(curr_best_case_revenue)
        best_case_pred_runtime = best_case_pred_runtime + curr_best_case_runtime
        #best_case_prob_lst.append(curr_bestcase_prob)
    
    robust_revenue_A = np.min(robust_revenue_lst)
    best_case_revenue_A = np.max(best_case_revenue_lst)
    
    ''' robust_prob_A = robust_prob_lst[np.argmin(robust_revenue_lst)]
    bestcase_prob_A = best_case_prob_lst[np.argmax(best_case_revenue_lst)] '''
    
    print(f'the robust revenue for assortment {assortment_A} is {robust_revenue_A} with runtime {robust_pred_runtime}')
    if robust_revenue_A>100:
        print('robust MDM prediction error ')
    print(f'the best-case revenue for assortment {assortment_A} is {best_case_revenue_A} with runtime {best_case_pred_runtime}\n')
    if best_case_revenue_A<0:
        print('best-case MDM prediction error ')
    
    ''' lb_sales_A = np.zeros(len(price))
    ub_sales_A = np.zeros(len(price))
    
    for k in range(len(assortment_A)):
        unit_price = np.zeros(n)
        unit_price[assortment_A[k]] = 1
        
        robust_product_prob_lst = []
        bestcase_product_prob_lst = []
        
        for j in range(len(lambda_A_lst)):
            if best_case_revenue_lst[j]>0:
                lambda_A = lambda_A_lst[j]
                curr_robust_prob = robust_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,unit_price)[0]
                
                robust_product_prob_lst.append(curr_robust_prob)
                
                curr_bestcase_prob = best_case_prediction_lp(curr_lam,lambda_A,prod_in_ass,assortment_A,curr_limit_prob,unit_price)[0]
                bestcase_product_prob_lst.append(curr_bestcase_prob)
        
        lb_sales_A[assortment_A[k]] = np.min(robust_product_prob_lst)
        ub_sales_A[assortment_A[k]] = np.max(bestcase_product_prob_lst)
        print(f'the sales lb for product {assortment_A[k]} in assortment {assortment_A} is {lb_sales_A[assortment_A[k]]}')
        print(f'the sales ub for product {assortment_A[k]} in assortment {assortment_A} is {ub_sales_A[assortment_A[k]]}\n')
     '''
    #return [robust_revenue_A,robust_pred_runtime,robust_prob_A,lb_sales_A, best_case_revenue_A,best_case_pred_runtime, bestcase_prob_A,ub_sales_A]
    return [robust_revenue_A,robust_pred_runtime, best_case_revenue_A,best_case_pred_runtime]


In [23]:

def mdm_prediction(curr_test_collection,curr_test_instance,price,curr_limit_prob, curr_lam, curr_train_collection):
    ''' true assortment ranking '''
    #true_choice_prob = [] 
    true_revenue = []
    
    ''' lb_mdm_rev_prob = []
    ub_mdm_rev_prob = []
    
    lb_mdm_sales = []
    ub_mdm_sales = [] '''
    
    lb_mdm_revenue = [] 
    ub_mdm_revenue = []
    
    lb_mdm_runtime = []
    ub_mdm_runtime = []
    
    prod_in_ass = [[] for _ in range(len(price))]
    for i, assort in enumerate(curr_train_collection):
        for j in assort:
            prod_in_ass[j].append(i)
    
    dec_lam = np.abs(np.sort(-curr_lam)) # multipliers in decreasing order, probabilities in increasing order
    lambda_A_lst = [np.max(curr_lam)+1]
    for i in range(len(dec_lam)-1):
        lambda_A_lst.append((dec_lam[i] + dec_lam[i+1])/2)
    lambda_A_lst.append(np.min(curr_lam)-1)
    
    for k in range(len(curr_test_collection)):
            
            assortment_A = curr_test_collection[k]
            
            # true revenue and true choice probabilities for current testing assortment
            curr_revenue = np.dot(curr_test_instance[:,k],price)
            true_revenue.append(curr_revenue)
            #true_choice_prob.append(curr_test_instance[:,k])
            
            # MDM revenue prediction
            print(f'revenue prediction for unseen assortment {assortment_A}')
            mdm_prediction_result = mdm_revenue(curr_limit_prob, curr_lam, prod_in_ass,lambda_A_lst, assortment_A, price)
            
            # return [robust_revenue_A,robust_pred_runtime, best_case_revenue_A,best_case_pred_runtime]
            
            ## record the revenue result
            lb_mdm_revenue.append(mdm_prediction_result[0])
            ub_mdm_revenue.append(mdm_prediction_result[2])
            
            ## record the choice probability result
            ''' lb_prob = np.zeros(n)
            lb_prob_result = mdm_prediction_result[2]
            
            ub_prob = np.zeros(n)
            ub_prob_result = mdm_prediction_result[6]
            
            for l in range(len(assortment_A)):
                lb_prob[assortment_A[l]] = lb_prob_result[l]
                ub_prob[assortment_A[l]] = ub_prob_result[l]
            
            lb_mdm_rev_prob.append(lb_prob)
            ub_mdm_rev_prob.append(ub_prob) '''
            
            ## record the runtime result
            lb_mdm_runtime.append(mdm_prediction_result[1])
            ub_mdm_runtime.append(mdm_prediction_result[3])
            
            
            ## MDM choice probability interval prediction
            ''' lb_sales_frac = mdm_prediction_result[3]
            ub_sales_frac = mdm_prediction_result[-1] '''
            
            ''' lb_mdm_sales.append(lb_sales_frac)
            ub_mdm_sales.append(ub_sales_frac) '''
    return true_revenue,lb_mdm_revenue,ub_mdm_revenue,lb_mdm_runtime,ub_mdm_runtime       
    #return true_choice_prob,true_revenue,lb_mdm_rev_prob,ub_mdm_rev_prob,lb_mdm_sales,ub_mdm_sales,lb_mdm_revenue,ub_mdm_revenue,lb_mdm_runtime,ub_mdm_runtime

In [24]:
## record the limit loss of all instances
all_mdm_limit_loss = []
all_mdm_limit_prob = []
all_mdm_limit_lam = []
all_mdm_limit_runtime = []


for i in range(len(pred_offer_times_list)):
    
    ## define container for mdm limit 
    mdm_limit_loss_collection =[]
    mdm_limit_runtime_collection =[]
    mdm_limit_probability_collection =[]
    mdm_limit_lam_collection =[]
       
    for j in range(pred_instance_size[i]):
        print(f'testing instance with offertimes {pred_offer_times_list[i]} : {j}th ')
        
        curr_limit_result = mdm_limit(all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
        # return [model.objVal, model.Runtime,lam_values,x_values]
        curr_limit_prob = curr_limit_result[-1]
        mdm_limit_probability_collection.append(curr_limit_prob)
        mdm_limit_loss_collection.append(curr_limit_result[0])
        mdm_limit_runtime_collection.append(curr_limit_result[1])
        mdm_limit_lam_collection.append(curr_limit_result[2])
            
    
    ## for each train instance [i][j] limit loss result is a number
    all_mdm_limit_loss.append(mdm_limit_loss_collection)
    all_mdm_limit_runtime.append(mdm_limit_runtime_collection)
    
    ## for each train instance [i][j] limit probability is a matrix
    all_mdm_limit_prob.append(mdm_limit_probability_collection)
    
    all_mdm_limit_lam.append(mdm_limit_lam_collection)

testing instance with offertimes 20 : 0th 
Academic license - for non-commercial use only - expires 2024-12-10
Using license file /Users/autumn/gurobi.lic
testing instance with offertimes 20 : 1th 
testing instance with offertimes 20 : 2th 
testing instance with offertimes 20 : 3th 
testing instance with offertimes 20 : 4th 
testing instance with offertimes 20 : 5th 
testing instance with offertimes 20 : 6th 
testing instance with offertimes 20 : 7th 
testing instance with offertimes 20 : 8th 
testing instance with offertimes 20 : 9th 
testing instance with offertimes 20 : 10th 
testing instance with offertimes 20 : 11th 
testing instance with offertimes 20 : 12th 
testing instance with offertimes 20 : 13th 
testing instance with offertimes 20 : 14th 
testing instance with offertimes 20 : 15th 
testing instance with offertimes 20 : 16th 
testing instance with offertimes 20 : 17th 
testing instance with offertimes 20 : 18th 
testing instance with offertimes 20 : 19th 
testing instance w

In [80]:
all_mdm_avg_loss  = []
for i in range(len(pred_offer_times_list)):
    mdm_avg_loss_collection = []
    
    for j in range(pred_instance_size[i]):
        mdm_avg_loss_collection.append(all_mdm_limit_loss[i][j]/np.sum(all_full_offertimes[i]))
    df_mdm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_loss':all_mdm_limit_loss[i],'avg_mdm_loss':mdm_avg_loss_collection,'mdm_limit_time':all_mdm_limit_runtime[i]})
    df_mdm_loss.to_csv('limit/mdm/limit/'+str(pred_offer_times_list[i])+'.csv')
    all_mdm_avg_loss.append(mdm_avg_loss_collection)

In [25]:
import pyomo.environ as pyo
from pyomo.environ import *

In [26]:
def mdm_kl_w_lam(lam,data,collection,offertimes):
    
    n, m = data.shape
    
    ind = [[] for i in range(n)]  ## keeps track of the assortments including product i

    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i)  ## assortment i includes product j
    
    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(n), range(m), domain= pyo.NonNegativeReals)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j not in collection[i]:
                model.norm_constraint.add(model.x[j, i] == 0)
    
    model.mdm_constraint = pyo.ConstraintList()
    for i in range(n):
        for j in range(len(ind[i]) - 1):
            for k in range(j + 1, len(ind[i])):
                if lam[ind[i][j]] >= lam[ind[i][k]]:
                    model.mdm_constraint.add(model.x[i, ind[i][k]] - model.x[i, ind[i][j]]  >= 0)
                if lam[ind[i][j]] <= lam[ind[i][k]]:
                    model.mdm_constraint.add(model.x[i, ind[i][j]] - model.x[i, ind[i][k]]  >= 0)
    
    model.obj = pyo.Objective(expr= -sum( sum(offertimes[j] *data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m)),sense=pyo.minimize)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    
    try:
        # Your optimization solve code here
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            print("Solver found a solution.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            return [pyo.value(model.obj), x_values, results.solver.time]
        
        elif results.solver.status == SolverStatus.warning:
            print("Solver issued a warning.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            return [pyo.value(model.obj), x_values, results.solver.time]
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")
    
    except ValueError as e:
        print(f"Error: {e}")
        
    

In [27]:
def deterministic_term(data,offertimes):
    dt = 0 
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            if data[i][j] >0:
                dt = dt + offertimes[j]*data[i][j]*np.log(data[i][j])
    return dt,dt/np.sum(offertimes)

In [38]:
all_mdm_kl_loss = []
all_mdm_avg_kl_loss = []
all_mdm_kl_prob = []
all_mdm_2stage_time = []

all_dt = []
all_avg_dt = []

all_total_time = []

for i in range(len(pred_offer_times_list)):
    
    mdm_kl_loss_collection = []
    mdm_avg_kl_loss_collection = []
    dt_collection = []
    avg_dt_collection = []
    mdm_kl_prob_collection = []
    mdm_2stage_time_collection = []
    total_time_collection = []
    
    for j in range(pred_instance_size[i]):
        result = mdm_kl_w_lam(all_mdm_limit_lam[i][j],all_full_instances[i][j],all_full_collections[i],all_full_offertimes[i])
        # result = [pyo.value(model.obj), x_values, results.solver.time]
        dt,avg_dt = deterministic_term(all_full_instances[i][j],all_full_offertimes[i])
        
        dt_collection.append(dt)
        avg_dt_collection.append(avg_dt)
        
        mdm_kl_loss_collection.append(result[0]+dt)
        mdm_avg_kl_loss_collection.append(result[0]/np.sum(all_full_offertimes[i])+avg_dt)
        
        mdm_kl_prob_collection.append(result[1])
        mdm_2stage_time_collection.append(result[-1])
        
        total_time_collection.append(result[-1] + all_mdm_limit_runtime[i][j])
        
    df_mdm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_kl_loss':mdm_kl_loss_collection,'avg_mdm_kl_loss':mdm_avg_kl_loss_collection,'mdm_kl_time':total_time_collection})
    df_mdm_loss.to_csv('limit/mdm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')

    all_mdm_kl_loss.append(mdm_kl_loss_collection) 
    all_mdm_avg_kl_loss.append(mdm_avg_kl_loss_collection)
    all_mdm_kl_prob.append(mdm_kl_prob_collection)
    all_total_time.append(total_time_collection)
        
    all_dt.append(dt_collection)
    all_avg_dt.append(avg_dt_collection)
    

Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      261
Number of nonzeros in inequality constraint Jacobian.:     1116
Number of nonzeros in Lagrangian Hessian.............:       67

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

In [39]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

avg_runtime = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mdm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mdm_kl_loss[i])/np.sqrt(len(all_mdm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mdm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mdm_avg_kl_loss[i])/np.sqrt(len(all_mdm_avg_kl_loss[i])))
    
    avg_runtime.append(np.mean(all_total_time[i]))
    
df_mdm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mdm_kl_loss':avg_total_kl_loss,'total_mdm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mdm_kl_loss':avg_avg_kl_loss,'avg_mdm_kl_loss_se':avg_avg_kl_loss_se,
                                   'mdm_kl_time':avg_runtime
                                   })
df_mdm_kl.to_csv('limit/mdm/mdm_kl_summary.csv')
df_mdm_kl 
    

,offertimes,total_mdm_kl_loss,total_mdm_kl_loss_se,avg_mdm_kl_loss,avg_mdm_kl_loss_se,mdm_kl_time
0,20,21.685475,1.998058,0.002833,0.000261,31.148317
1,30,17.620152,2.756008,0.002340,0.000366,4.576694
2,40,13.644756,2.888305,0.001854,0.000392,1.530094
3,50,6.155370,1.654600,0.000856,0.000230,0.722602
4,60,6.204588,1.798703,0.000876,0.000254,0.571131


In [54]:
for i in range(len(pred_offer_times_list)):
    df_dt = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'dt':all_dt[i],'avg_dt':all_avg_dt[i]})
    df_dt.to_csv('limit/dt/'+str(pred_offer_times_list[i])+'.csv')

In [59]:
df_mnl_kl = pd.read_csv('limit/mnl/mnl_kl_summary.csv')
df_mnl_kl  = df_mnl_kl.drop(df_mnl_kl.columns[0],axis=1)
df_mnl_kl

,offertimes,total_mnl_kl_loss,total_mnl_kl_loss_se,avg_mnl_kl_loss,avg_mnl_kl_loss_se
0,20,148.757021,3.655840,0.019433,0.000478
1,30,130.229694,3.671666,0.017297,0.000488
2,40,109.435407,3.308701,0.014867,0.000449
3,50,88.610340,3.079184,0.012324,0.000428
4,60,75.923897,2.980402,0.010721,0.000421


In [58]:
df_lcmnl_kl = pd.read_csv('limit/lcmnl/lcmnl_kl_summary.csv')
df_lcmnl_kl = df_lcmnl_kl.drop(df_lcmnl_kl.columns[0],axis=1)
df_lcmnl_kl

,offertimes,total_lcmnl_kl_loss,total_lcmnl_kl_loss_se,avg_lcmnl_kl_loss,avg_lcmnl_kl_loss_se
0,20,10.204440,0.501781,0.001333,0.000066
1,30,6.224155,0.363629,0.000827,0.000048
2,40,4.151991,0.371206,0.000564,0.000050
3,50,3.408609,1.208478,0.000474,0.000168
4,60,2.483311,0.870619,0.000351,0.000123


In [61]:
# Merge the dataframes on the 'offertimes' column
merged_df = df_mdm_kl.merge(df_mnl_kl, on='offertimes').merge(df_lcmnl_kl, on='offertimes')

# Display the merged dataframe
merged_df.to_csv('limit/kl_summary.csv')

,offertimes,total_mdm_kl_loss,total_mdm_kl_loss_se,avg_mdm_kl_loss,avg_mdm_kl_loss_se,mdm_kl_time,total_mnl_kl_loss,total_mnl_kl_loss_se,avg_mnl_kl_loss,avg_mnl_kl_loss_se,total_lcmnl_kl_loss,total_lcmnl_kl_loss_se,avg_lcmnl_kl_loss,avg_lcmnl_kl_loss_se
0,20,21.685475,1.998058,0.002833,0.000261,31.148317,148.757021,3.655840,0.019433,0.000478,10.204440,0.501781,0.001333,0.000066
1,30,17.620152,2.756008,0.002340,0.000366,4.576694,130.229694,3.671666,0.017297,0.000488,6.224155,0.363629,0.000827,0.000048
2,40,13.644756,2.888305,0.001854,0.000392,1.530094,109.435407,3.308701,0.014867,0.000449,4.151991,0.371206,0.000564,0.000050
3,50,6.155370,1.654600,0.000856,0.000230,0.722602,88.610340,3.079184,0.012324,0.000428,3.408609,1.208478,0.000474,0.000168
4,60,6.204588,1.798703,0.000876,0.000254,0.571131,75.923897,2.980402,0.010721,0.000421,2.483311,0.870619,0.000351,0.000123


In [63]:
avg_kl_loss_summary = pd.concat([merged_df['offertimes'],
                                merged_df['avg_mdm_kl_loss'],merged_df['avg_mdm_kl_loss_se'],merged_df['mdm_kl_time'],
                                merged_df['avg_mnl_kl_loss'],merged_df['avg_mnl_kl_loss_se'],
                                merged_df['avg_lcmnl_kl_loss'],merged_df['avg_lcmnl_kl_loss_se']]
                                ,axis=1)
avg_kl_loss_summary.to_csv('limit/avg_kl_summary.csv')

In [65]:
avg_kl_loss_summary

,offertimes,avg_mdm_kl_loss,avg_mdm_kl_loss_se,mdm_kl_time,avg_mnl_kl_loss,avg_mnl_kl_loss_se,avg_lcmnl_kl_loss,avg_lcmnl_kl_loss_se
0,20,0.002833,0.000261,31.148317,0.019433,0.000478,0.001333,0.000066
1,30,0.002340,0.000366,4.576694,0.017297,0.000488,0.000827,0.000048
2,40,0.001854,0.000392,1.530094,0.014867,0.000449,0.000564,0.000050
3,50,0.000856,0.000230,0.722602,0.012324,0.000428,0.000474,0.000168
4,60,0.000876,0.000254,0.571131,0.010721,0.000421,0.000351,0.000123


In [64]:
## adding limit results
ins_idx = list(range(pred_instance_size[0]))
for i in range(len(pred_offer_times_list)):
    for j in range(pred_instance_size[i]):
        df_limit_prob = pd.DataFrame(all_mdm_kl_prob[i][j])
        df_limit_prob.to_csv('limit/mdm/limit_kl/limit_kl_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')

In [66]:
all_mnl_kl_loss = []
all_mnl_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mnl_kl_loss_collection = []
    mnl_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        mnl_prob = pd.read_csv('limit/mnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mnl_prob = mnl_prob.drop(mnl_prob.columns[0],axis=1)
        limit_prob = np.array(mnl_prob)
        
        #limit_prob = all_mnl_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        mnl_kl_loss_collection.append(-curr_kl_loss)
        mnl_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    #df_mnl_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mnl_loss':mnl_kl_loss_collection,'avg_loss':mnl_avg_kl_loss_collection})
    #df_mnl_loss.to_csv('limit/mnl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mnl_kl_loss.append(mnl_kl_loss_collection) 
    all_mnl_avg_kl_loss.append(mnl_avg_kl_loss_collection)
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mnl_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mnl_kl_loss[i])/np.sqrt(len(all_mnl_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mnl_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mnl_avg_kl_loss[i])/np.sqrt(len(all_mnl_avg_kl_loss[i])))
    
df_mnl_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mnl_kl_loss':avg_total_kl_loss,'total_mnl_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mnl_kl_loss':avg_avg_kl_loss,'avg_mnl_kl_loss_se':avg_avg_kl_loss_se
                                   })
#df_mnl_kl.to_csv('limit/mnl/mnl_kl_summary.csv')
df_mnl_kl 

,offertimes,total_mnl_kl_loss,total_mnl_kl_loss_se,avg_mnl_kl_loss,avg_mnl_kl_loss_se
0,20,148.757021,3.655840,0.019433,0.000478
1,30,130.229694,3.671666,0.017297,0.000488
2,40,109.435407,3.308701,0.014867,0.000449
3,50,88.610340,3.079184,0.012324,0.000428
4,60,75.923897,2.980402,0.010721,0.000421


In [67]:
all_lcmnl_kl_loss = []
all_lcmnl_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    lcmnl_kl_loss_collection = []
    lcmnl_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        lcmnl_prob = pd.read_csv('limit/lcmnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        lcmnl_prob = lcmnl_prob.drop(lcmnl_prob.columns[0],axis=1)
        limit_prob = np.array(lcmnl_prob)
        
        #limit_prob = all_lcmnl_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        lcmnl_kl_loss_collection.append(-curr_kl_loss)
        lcmnl_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    #df_lcmnl_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'lcmnl_loss':lcmnl_kl_loss_collection,'avg_loss':lcmnl_avg_kl_loss_collection})
    #df_lcmnl_loss.to_csv('limit/lcmnl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_lcmnl_kl_loss.append(lcmnl_kl_loss_collection) 
    all_lcmnl_avg_kl_loss.append(lcmnl_avg_kl_loss_collection)

avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_lcmnl_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_lcmnl_kl_loss[i])/np.sqrt(len(all_lcmnl_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_lcmnl_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_lcmnl_avg_kl_loss[i])/np.sqrt(len(all_lcmnl_avg_kl_loss[i])))
    
df_lcmnl_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_lcmnl_kl_loss':avg_total_kl_loss,'total_lcmnl_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_lcmnl_kl_loss':avg_avg_kl_loss,'avg_lcmnl_kl_loss_se':avg_avg_kl_loss_se
                                   })
#df_lcmnl_kl.to_csv('limit/lcmnl/lcmnl_kl_summary.csv')
df_lcmnl_kl 

,offertimes,total_lcmnl_kl_loss,total_lcmnl_kl_loss_se,avg_lcmnl_kl_loss,avg_lcmnl_kl_loss_se
0,20,10.204440,0.501781,0.001333,0.000066
1,30,6.224155,0.363629,0.000827,0.000048
2,40,4.151991,0.371206,0.000564,0.000050
3,50,3.408609,1.208478,0.000474,0.000168
4,60,2.483311,0.870619,0.000351,0.000123


In [70]:
# independet model kl loss

all_indep_model_kl_loss = []
all_indep_model_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    indep_model_kl_loss_collection = []
    indep_model_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]): 
        idep_model_kl = 0 
        for k in range(len(all_full_collections[i])):
            assortment = all_full_collections[i][k]
            for x in assortment:
                if all_full_instances[i][j][x][k]>0:
                    idep_model_kl = idep_model_kl - all_full_offertimes[i][k]*all_full_instances[i][j][x][k]*np.log((1/len(assortment))/all_full_instances[i][j][x][k])
                    
        indep_model_kl_loss_collection.append(idep_model_kl)
        indep_model_avg_kl_loss_collection.append(idep_model_kl/np.sum(all_full_offertimes[i]))
    df_indep_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'indep_kl_loss':indep_model_kl_loss_collection,'avg_indep_kl_loss':indep_model_avg_kl_loss_collection})
    df_indep_loss.to_csv('limit/indep_model/'+str(pred_offer_times_list[i])+'.csv')
        
    all_indep_model_kl_loss.append(indep_model_kl_loss_collection)
    all_indep_model_avg_kl_loss.append(indep_model_avg_kl_loss_collection)
    
avg_total_indep_kl_loss = []
avg_total_indep_kl_loss_se = []
avg_avg_indep_kl_loss = []
avg_avg_indep_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_indep_kl_loss.append(np.mean(all_indep_model_kl_loss[i]))
    avg_total_indep_kl_loss_se.append(np.std(all_indep_model_kl_loss[i])/np.sqrt(len(all_indep_model_kl_loss[i])))
    
    avg_avg_indep_kl_loss.append(np.mean(all_indep_model_avg_kl_loss[i]))
    avg_avg_indep_kl_loss_se.append(np.std(all_indep_model_avg_kl_loss[i])/np.sqrt(len(all_indep_model_avg_kl_loss[i])))
    
df_indep_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_indep_kl_loss':avg_total_indep_kl_loss,'total_indep_kl_loss_se':avg_total_indep_kl_loss_se,
                                   'avg_indep_kl_loss':avg_avg_indep_kl_loss,'avg_indep_kl_loss_se':avg_avg_indep_kl_loss_se
                                   })
df_indep_kl.to_csv('limit/indep_model/indep_kl_summary.csv')
df_indep_kl         

,offertimes,total_indep_kl_loss,total_indep_kl_loss_se,avg_indep_kl_loss,avg_indep_kl_loss_se
0,20,1103.553472,27.082538,0.144161,0.003538
1,30,1045.741526,26.559974,0.138895,0.003528
2,40,978.886269,25.548457,0.132983,0.003471
3,50,912.659415,24.753265,0.126935,0.003443
4,60,871.867775,24.344689,0.123110,0.003438


In [74]:
# compute the improvement 
# report loss redution compared to independent model
all_mdm_percent = []
all_mnl_percent = []
all_lcmnl_percent = []

mdm_mean_lst = []
mnl_mean_lst = []
lcmnl_mean_lst = []

mdm_se_lst = []
mnl_se_lst = []
lcmnl_se_lst = []

for i in range(len(pred_offer_times_list)): 
    mdm_percent_collection = []
    mnl_percent_collection = []
    lcmnl_percent_collection = []
    for j in range(pred_instance_size[i]): 
        mdm_percent_collection.append(1-(all_mdm_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        mnl_percent_collection.append(1-(all_mnl_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        lcmnl_percent_collection.append(1-(all_lcmnl_avg_kl_loss[i][j]/all_indep_model_avg_kl_loss[i][j]))
        
    df_percent = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_percent':mdm_percent_collection,'mnl_percent':mnl_percent_collection,'lcmnl_percent':lcmnl_percent_collection})
    df_percent.to_csv('limit/kl_compare/'+str(pred_offer_times_list[i])+'.csv')
    
    mdm_mean_lst.append(np.mean(mdm_percent_collection))
    mnl_mean_lst.append(np.mean(mnl_percent_collection))
    lcmnl_mean_lst.append(np.mean(lcmnl_percent_collection))
    
    mdm_se_lst.append(np.std(mdm_percent_collection)/np.sqrt(len(mdm_percent_collection)))
    mnl_se_lst.append(np.std(mnl_percent_collection)/np.sqrt(len(mnl_percent_collection)))
    lcmnl_se_lst.append(np.std(lcmnl_percent_collection)/np.sqrt(len(lcmnl_percent_collection)))

df_percent_summary = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'mdm_percent':mdm_mean_lst,'mdm_se':mdm_se_lst,
                                   'mnl_percent':mnl_mean_lst,'mnl_se':mnl_se_lst,
                                   'lcmnl_percent':lcmnl_mean_lst,'lcmnl_se':lcmnl_se_lst})
df_percent_summary.to_csv('limit/kl_compare/improve_percent_summary.csv')
df_percent_summary

,offertimes,mdm_percent,mdm_se,mnl_percent,mnl_se,lcmnl_percent,lcmnl_se
0,20,0.979882,0.001909,0.863366,0.003393,0.990621,0.000450
1,30,0.983034,0.002538,0.873957,0.003409,0.994073,0.000311
2,40,0.986082,0.002868,0.886992,0.003145,0.995852,0.000316
3,50,0.993550,0.001577,0.901991,0.003063,0.995974,0.001651
4,60,0.992773,0.002133,0.912301,0.003023,0.997143,0.001046
